In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [ ]:
import unicodedata, re
from tqdm.notebook import tqdm

In [ ]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(insta_url)
time.sleep(3)

### 로그인하기

In [ ]:
with open('password.txt') as f:
    password = f.read()

In [ ]:
id_ = 'vvbshvv'
input_id_=driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_id_.clear()
input_id_.send_keys(id_)

In [ ]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)

In [ ]:
input_pwd.submit()
time.sleep(2)

In [ ]:
#로그인 정보 저장 - 나중에하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [ ]:
#알림설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 제주맛집으로 검색

In [ ]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%A7%9B%EC%A7%91'

In [ ]:
driver.get(url)
time.sleep(5)

In [ ]:
# 첫번째 게시글 클릭
driver.find_element_by_css_selector('._9AhH0').click()
time.sleep(2)


In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 본문내용가져오기
try:
    content = soup.select_one('.C4VMK > span').text
    content = unicodedata.normalize('NFC',content)
except:
    content = ' '
content

'제주 갈치요리하면 요기 !!!비주얼 최고맛집이라불리는 갈치요리맛집"성산을본갈치"성산과 우도를 한눈에 볼수있는 뷰맛집 🏞문어 전복 딱새우 갈치 등등싱싱하고 다양한 해산물이 넘치도록 쏟아진다🦀🐙🦐🐟🦞🦑비주얼 끝판왕 성산 찐맛집 😋살이통통한 갈치를 맛보시려면성산을본갈치로 GoGoGo~~~❗상호 : #성산을본갈치주소 : 서귀포시 성산중앙로64번길5-4 2층영업시간 : 연중무휴 09:00~21:00전화 : 064 - 782 - 6868'

In [ ]:
# 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+',content)
tags

['#성산을본갈치주소']

In [ ]:
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-07'

In [ ]:
# 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [ ]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [ ]:
row = [content, date, like, place, tags]

In [ ]:
#첫번째 게시글에서 다음글로이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
time.sleep(2)

In [ ]:
# 두번째 게시글에서 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

In [ ]:
# pop up화면을 없앰(x를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

In [ ]:
# 6을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [ ]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [86]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]